In [91]:
import lamda
import numpy as np
import pandas as pd
import ast

movies = pd.read_csv('movies.csv')
credits = pd.read_csv('credits.csv')

In [92]:
movies = movies.merge(credits, on='title')

In [93]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [94]:
movies.dropna(inplace=True)

In [95]:
def convert(obj):
    templist=[]
    for i in ast.literal_eval(obj):
        templist.append(i['name'])
    return templist

In [96]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [97]:
def convert3(obj):
    templist=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            templist.append(i['name'])
            counter += 1
        else:
            break
    return templist
    
movies['cast'] = movies['cast'].apply(convert3)

In [98]:
def fetch_director(obj):
    templist=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            templist.append(i['name'])
    return templist
    
movies['crew'] = movies['crew'].apply(fetch_director)

In [99]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [100]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [102]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [104]:
movies2 = movies[['movie_id', 'title', 'tags']]

In [106]:
movies2['tags'] = movies2['tags'].apply(lambda x:' '.join(x))

C:\Users\heatb\AppData\Local\Temp\ipykernel_72028\221509110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies2['tags'] = movies2['tags'].apply(lambda x:' '.join(x))


In [109]:
movies2['tags'] = movies2['tags'].apply(lambda X:X.lower())

C:\Users\heatb\AppData\Local\Temp\ipykernel_72028\110294168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies2['tags'] = movies2['tags'].apply(lambda X:X.lower())


In [111]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [114]:
vectors = cv.fit_transform(movies2['tags']).toarray()

In [115]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [116]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [117]:
movies2['tags'] = movies2['tags'].apply(stem)

C:\Users\heatb\AppData\Local\Temp\ipykernel_72028\4172303934.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies2['tags'] = movies2['tags'].apply(stem)


In [118]:
from sklearn.metrics.pairwise import cosine_similarity

In [119]:
similarity = cosine_similarity(vectors)

In [120]:
sorted(list(enumerate(similarity[0])), key=lambda x:x[1], reverse=True)[1:6]

[(539, np.float64(0.25724787771376323)),
 (1192, np.float64(0.24873416908154544)),
 (260, np.float64(0.24759378423606915)),
 (1214, np.float64(0.24595492912420727)),
 (507, np.float64(0.24498947175305572))]

In [121]:
def recommend(movie):
    movie_index = movies2[movies2['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), key=lambda x:x[1], reverse=True)[1:6]
    
    for i in movies_list:
        print(movies2.iloc[i[0]].title)

In [125]:
recommend('National Treasure')

Lara Croft: Tomb Raider
Indiana Jones and the Kingdom of the Crystal Skull
Lara Croft Tomb Raider: The Cradle of Life
Raiders of the Lost Ark
Indiana Jones and the Last Crusade
